In [9]:
import psycopg
import time
import pandas as pd

# basic join example
also works for left / right join

## inefficient

In [24]:
%%time
user = 'postgres'
host = 'localhost'
dbname = 'msds691_HW'

with psycopg.connect(f"user='{user}' \
                         host='{host}' \
                         dbname='{dbname}'") as conn:
        with conn.cursor() as curs:
            query = \
            '''
       SELECT
    i.id,
    i.incident_datetime,
    i.report_datetime,
    i.longitude,
    i.latitude,
    (SELECT it.incident_category FROM incident_type it WHERE it.incident_code = i.incident_code) AS incident_category,
    (SELECT it.incident_subcategory FROM incident_type it WHERE it.incident_code = i.incident_code) AS incident_subcategory
        FROM incident i;
            '''
            results = curs.execute(query).fetchall() 

CPU times: user 508 ms, sys: 121 ms, total: 629 ms
Wall time: 1.4 s


In [25]:
pd.DataFrame(results, columns=['id', 'incident_datetime', 'report_datetime', 'longitude', 'latitude', 'incident_category', 'incident_subcategory'])

,id,incident_datetime,report_datetime,longitude,latitude,incident_category,incident_subcategory
0,704938,2018-08-18 14:00:00,2018-08-18 17:19:00,-122.439490,37.791843,Larceny Theft,Larceny - From Vehicle
1,704939,2018-07-14 00:00:00,2018-08-16 21:38:00,-122.399150,37.795162,Lost Property,Lost Property
2,704940,2018-08-16 22:00:00,2018-08-17 02:08:00,-122.410500,37.806960,Larceny Theft,Larceny - From Vehicle
3,704941,2018-08-16 14:20:00,2018-08-18 08:03:00,-122.429980,37.786503,Larceny Theft,Larceny - From Vehicle
4,704942,2018-08-18 03:00:00,2018-08-18 21:59:00,-122.424860,37.786198,Larceny Theft,Larceny - From Vehicle
...,...,...,...,...,...,...,...
637437,704933,2018-08-11 05:30:00,2018-08-16 21:35:00,-122.500725,37.747560,Larceny Theft,Larceny - From Vehicle
637438,704934,2018-08-18 09:30:00,2018-08-19 13:44:00,-122.492256,37.764780,Larceny Theft,Larceny - From Vehicle
637439,704935,2018-08-16 22:00:00,2018-08-17 01:08:00,-122.397940,37.790054,Larceny Theft,Larceny Theft - Other
637440,704936,2018-08-16 20:15:00,2018-08-17 00:34:00,-122.403620,37.801144,Larceny Theft,Larceny - From Vehicle


## efficient

In [26]:
%%time

user = 'postgres'
host = 'localhost'
dbname = 'msds691_HW'

with psycopg.connect(f"user='{user}' \
                         host='{host}' \
                         dbname='{dbname}'") as conn:
        with conn.cursor() as curs:
            query = \
            '''
		SELECT
    i.id,
    i.incident_datetime,
    i.report_datetime,
    i.longitude,
    i.latitude,
    it.incident_category,
    it.incident_subcategory
FROM incident i
 JOIN report_type rt ON i.report_type_code = rt.report_type_code
 JOIN incident_type it ON i.incident_code = it.incident_code;
            '''
            a = curs.execute(query).fetchall() 

CPU times: user 473 ms, sys: 107 ms, total: 580 ms
Wall time: 771 ms


In [27]:
pd.DataFrame(a, columns=['id', 'incident_datetime', 'report_datetime', 'longitude', 'latitude', 'incident_category', 'incident_subcategory'])

,id,incident_datetime,report_datetime,longitude,latitude,incident_category,incident_subcategory
0,704938,2018-08-18 14:00:00,2018-08-18 17:19:00,-122.439490,37.791843,Larceny Theft,Larceny - From Vehicle
1,704939,2018-07-14 00:00:00,2018-08-16 21:38:00,-122.399150,37.795162,Lost Property,Lost Property
2,704940,2018-08-16 22:00:00,2018-08-17 02:08:00,-122.410500,37.806960,Larceny Theft,Larceny - From Vehicle
3,704941,2018-08-16 14:20:00,2018-08-18 08:03:00,-122.429980,37.786503,Larceny Theft,Larceny - From Vehicle
4,704942,2018-08-18 03:00:00,2018-08-18 21:59:00,-122.424860,37.786198,Larceny Theft,Larceny - From Vehicle
...,...,...,...,...,...,...,...
637437,704933,2018-08-11 05:30:00,2018-08-16 21:35:00,-122.500725,37.747560,Larceny Theft,Larceny - From Vehicle
637438,704934,2018-08-18 09:30:00,2018-08-19 13:44:00,-122.492256,37.764780,Larceny Theft,Larceny - From Vehicle
637439,704935,2018-08-16 22:00:00,2018-08-17 01:08:00,-122.397940,37.790054,Larceny Theft,Larceny Theft - Other
637440,704936,2018-08-16 20:15:00,2018-08-17 00:34:00,-122.403620,37.801144,Larceny Theft,Larceny - From Vehicle


# inner join example

## inefficient

In [28]:
%%time

user = 'postgres'
host = 'localhost'
dbname = 'msds691_HW'

with psycopg.connect(f"user='{user}' \
                         host='{host}' \
                         dbname='{dbname}'") as conn:
        with conn.cursor() as curs:
            query = \
            '''
		SELECT
    i.id,
    i.incident_datetime,
    i.report_datetime,
    i.longitude,
    i.latitude,
    (SELECT it.incident_category FROM incident_type it WHERE it.incident_code = i.incident_code) AS incident_category,
    (SELECT it.incident_subcategory FROM incident_type it WHERE it.incident_code = i.incident_code) AS incident_subcategory
FROM incident i
WHERE EXISTS (
    SELECT 1 FROM report_type rt WHERE rt.report_type_code = i.report_type_code
)
AND EXISTS (
    SELECT 1 FROM incident_type it WHERE it.incident_code = i.incident_code
);

            '''
            a = curs.execute(query).fetchall() 

CPU times: user 556 ms, sys: 107 ms, total: 664 ms
Wall time: 1.52 s


In [29]:
pd.DataFrame(a, columns=['id', 'incident_datetime', 'report_datetime', 'longitude', 'latitude', 'incident_category', 'incident_subcategory'])

,id,incident_datetime,report_datetime,longitude,latitude,incident_category,incident_subcategory
0,704938,2018-08-18 14:00:00,2018-08-18 17:19:00,-122.439490,37.791843,Larceny Theft,Larceny - From Vehicle
1,704939,2018-07-14 00:00:00,2018-08-16 21:38:00,-122.399150,37.795162,Lost Property,Lost Property
2,704940,2018-08-16 22:00:00,2018-08-17 02:08:00,-122.410500,37.806960,Larceny Theft,Larceny - From Vehicle
3,704941,2018-08-16 14:20:00,2018-08-18 08:03:00,-122.429980,37.786503,Larceny Theft,Larceny - From Vehicle
4,704942,2018-08-18 03:00:00,2018-08-18 21:59:00,-122.424860,37.786198,Larceny Theft,Larceny - From Vehicle
...,...,...,...,...,...,...,...
637437,704933,2018-08-11 05:30:00,2018-08-16 21:35:00,-122.500725,37.747560,Larceny Theft,Larceny - From Vehicle
637438,704934,2018-08-18 09:30:00,2018-08-19 13:44:00,-122.492256,37.764780,Larceny Theft,Larceny - From Vehicle
637439,704935,2018-08-16 22:00:00,2018-08-17 01:08:00,-122.397940,37.790054,Larceny Theft,Larceny Theft - Other
637440,704936,2018-08-16 20:15:00,2018-08-17 00:34:00,-122.403620,37.801144,Larceny Theft,Larceny - From Vehicle


## efficient

In [30]:
%%time

user = 'postgres'
host = 'localhost'
dbname = 'msds691_HW'

with psycopg.connect(f"user='{user}' \
                         host='{host}' \
                         dbname='{dbname}'") as conn:
        with conn.cursor() as curs:
            query = \
            '''
		SELECT
    i.id,
    i.incident_datetime,
    i.report_datetime,
    i.longitude,
    i.latitude,
    it.incident_category,
    it.incident_subcategory
FROM incident i
JOIN report_type rt ON i.report_type_code = rt.report_type_code
JOIN incident_type it ON i.incident_code = it.incident_code;

            '''
            a = curs.execute(query).fetchall() 

CPU times: user 544 ms, sys: 103 ms, total: 647 ms
Wall time: 1.05 s


In [31]:
pd.DataFrame(a, columns=['id', 'incident_datetime', 'report_datetime', 'longitude', 'latitude', 'incident_category', 'incident_subcategory'])

,id,incident_datetime,report_datetime,longitude,latitude,incident_category,incident_subcategory
0,704938,2018-08-18 14:00:00,2018-08-18 17:19:00,-122.439490,37.791843,Larceny Theft,Larceny - From Vehicle
1,704939,2018-07-14 00:00:00,2018-08-16 21:38:00,-122.399150,37.795162,Lost Property,Lost Property
2,704940,2018-08-16 22:00:00,2018-08-17 02:08:00,-122.410500,37.806960,Larceny Theft,Larceny - From Vehicle
3,704941,2018-08-16 14:20:00,2018-08-18 08:03:00,-122.429980,37.786503,Larceny Theft,Larceny - From Vehicle
4,704942,2018-08-18 03:00:00,2018-08-18 21:59:00,-122.424860,37.786198,Larceny Theft,Larceny - From Vehicle
...,...,...,...,...,...,...,...
637437,704933,2018-08-11 05:30:00,2018-08-16 21:35:00,-122.500725,37.747560,Larceny Theft,Larceny - From Vehicle
637438,704934,2018-08-18 09:30:00,2018-08-19 13:44:00,-122.492256,37.764780,Larceny Theft,Larceny - From Vehicle
637439,704935,2018-08-16 22:00:00,2018-08-17 01:08:00,-122.397940,37.790054,Larceny Theft,Larceny Theft - Other
637440,704936,2018-08-16 20:15:00,2018-08-17 00:34:00,-122.403620,37.801144,Larceny Theft,Larceny - From Vehicle


# outer join example
## inefficient

In [32]:
%%time

user = 'postgres'
host = 'localhost'
dbname = 'msds691_HW'

with psycopg.connect(f"user='{user}' \
                         host='{host}' \
                         dbname='{dbname}'") as conn:
        with conn.cursor() as curs:
            query = \
            '''
		SELECT
    i.id,
    i.incident_datetime,
    i.report_datetime,
    i.longitude,
    i.latitude,
    (SELECT it.incident_category FROM incident_type it WHERE it.incident_code = i.incident_code) AS incident_category,
    (SELECT it.incident_subcategory FROM incident_type it WHERE it.incident_code = i.incident_code) AS incident_subcategory
FROM incident i;

            '''
            a = curs.execute(query).fetchall() 

CPU times: user 546 ms, sys: 100 ms, total: 646 ms
Wall time: 1.41 s


In [33]:
pd.DataFrame(a, columns=['id', 'incident_datetime', 'report_datetime', 'longitude', 'latitude', 'incident_category', 'incident_subcategory'])

,id,incident_datetime,report_datetime,longitude,latitude,incident_category,incident_subcategory
0,704938,2018-08-18 14:00:00,2018-08-18 17:19:00,-122.439490,37.791843,Larceny Theft,Larceny - From Vehicle
1,704939,2018-07-14 00:00:00,2018-08-16 21:38:00,-122.399150,37.795162,Lost Property,Lost Property
2,704940,2018-08-16 22:00:00,2018-08-17 02:08:00,-122.410500,37.806960,Larceny Theft,Larceny - From Vehicle
3,704941,2018-08-16 14:20:00,2018-08-18 08:03:00,-122.429980,37.786503,Larceny Theft,Larceny - From Vehicle
4,704942,2018-08-18 03:00:00,2018-08-18 21:59:00,-122.424860,37.786198,Larceny Theft,Larceny - From Vehicle
...,...,...,...,...,...,...,...
637437,704933,2018-08-11 05:30:00,2018-08-16 21:35:00,-122.500725,37.747560,Larceny Theft,Larceny - From Vehicle
637438,704934,2018-08-18 09:30:00,2018-08-19 13:44:00,-122.492256,37.764780,Larceny Theft,Larceny - From Vehicle
637439,704935,2018-08-16 22:00:00,2018-08-17 01:08:00,-122.397940,37.790054,Larceny Theft,Larceny Theft - Other
637440,704936,2018-08-16 20:15:00,2018-08-17 00:34:00,-122.403620,37.801144,Larceny Theft,Larceny - From Vehicle


## efficient

In [34]:
%%time

user = 'postgres'
host = 'localhost'
dbname = 'msds691_HW'

with psycopg.connect(f"user='{user}' \
                         host='{host}' \
                         dbname='{dbname}'") as conn:
        with conn.cursor() as curs:
            query = \
            '''
		SELECT
    i.id,
    i.incident_datetime,
    i.report_datetime,
    i.longitude,
    i.latitude,
    it.incident_category,
    it.incident_subcategory
FROM incident i
LEFT JOIN report_type rt ON i.report_type_code = rt.report_type_code
LEFT JOIN incident_type it ON i.incident_code = it.incident_code;


            '''
            a = curs.execute(query).fetchall() 

CPU times: user 547 ms, sys: 104 ms, total: 651 ms
Wall time: 796 ms


In [35]:
pd.DataFrame(a, columns=['id', 'incident_datetime', 'report_datetime', 'longitude', 'latitude', 'incident_category', 'incident_subcategory'])

,id,incident_datetime,report_datetime,longitude,latitude,incident_category,incident_subcategory
0,704938,2018-08-18 14:00:00,2018-08-18 17:19:00,-122.439490,37.791843,Larceny Theft,Larceny - From Vehicle
1,704939,2018-07-14 00:00:00,2018-08-16 21:38:00,-122.399150,37.795162,Lost Property,Lost Property
2,704940,2018-08-16 22:00:00,2018-08-17 02:08:00,-122.410500,37.806960,Larceny Theft,Larceny - From Vehicle
3,704941,2018-08-16 14:20:00,2018-08-18 08:03:00,-122.429980,37.786503,Larceny Theft,Larceny - From Vehicle
4,704942,2018-08-18 03:00:00,2018-08-18 21:59:00,-122.424860,37.786198,Larceny Theft,Larceny - From Vehicle
...,...,...,...,...,...,...,...
637437,704933,2018-08-11 05:30:00,2018-08-16 21:35:00,-122.500725,37.747560,Larceny Theft,Larceny - From Vehicle
637438,704934,2018-08-18 09:30:00,2018-08-19 13:44:00,-122.492256,37.764780,Larceny Theft,Larceny - From Vehicle
637439,704935,2018-08-16 22:00:00,2018-08-17 01:08:00,-122.397940,37.790054,Larceny Theft,Larceny Theft - Other
637440,704936,2018-08-16 20:15:00,2018-08-17 00:34:00,-122.403620,37.801144,Larceny Theft,Larceny - From Vehicle
